#### In this notebook I start first from building a quick EDA revealing patters in the data, then I use EDA insights to build a simple linear model with only essential features (based on a out of time approach).

##### Since we will use Scikit-learn models, let's accelerate them first by using Intel Extension.

* vers. 20 - introduced selective rounding
* vers. 22 - adding sin/cos processing of day of the year at different time lags

In [ ]:
!pip install scikit-learn -U

In [ ]:
# Intel® Extension for Scikit-learn installation:
!pip install scikit-learn-intelex

In [ ]:
from sklearnex import patch_sklearn
patch_sklearn()

##### We then load the data, it is necessary paying attention to convert the date into datetime

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn

In [ ]:
# Loading train and test data
train = pd.read_csv("../input/tabular-playground-series-jan-2022/train.csv", parse_dates=['date'])
test = pd.read_csv("../input/tabular-playground-series-jan-2022/test.csv", parse_dates=['date'])

In [ ]:
train.dtypes

##### Before starting with EDA, it is important to check about the data structure. Apparently we have a combination of time series based on countries, stores and products. Let's first check if all the combinations appear in train and test.

In [ ]:
# figuring out the theoretically possible level combination
time_series = ['country', 'store', 'product']
combinations = 1
for feat in time_series:
    combinations *= train[feat].nunique()
    
print(f"There are {combinations} possible combinations")

In [ ]:
time_series = ['country', 'store', 'product']
country_store_product_train = train[time_series].drop_duplicates().sort_values(time_series)
country_store_product_test =test[time_series].drop_duplicates().sort_values(time_series)

cond_1 = len(country_store_product_train) == combinations
print(f"Are all theoretical combinations present in train: {cond_1}")
cond_2 = (country_store_product_train == country_store_product_test).all().all()
print(f"Are combinations the same in train and test: {cond_2}")

##### As a second step let's visualize how time is split between train and test.

In [ ]:
train_dates = train.date.drop_duplicates().sort_values()
test_dates = test.date.drop_duplicates().sort_values()

fig, ax = plt.subplots(1, 1, figsize = (11, 7))
cmap_cv = plt.cm.coolwarm

color_index = np.array([1] * len(train_dates) + [0] * len(test_dates))

ax.scatter(range(len(train_dates)), [.5] * len(train_dates),
           c=color_index[:len(train_dates)], marker='_', lw=15, cmap=cmap_cv,
           label='train', vmin=-.2, vmax=1.2)

ax.scatter(range(len(train_dates), len(train_dates) + len(test_dates)), [.55] * len(test_dates),
           c=color_index[len(train_dates):], marker='_', lw=15, cmap=cmap_cv,
           label='test', vmin=-.2, vmax=1.2)

tick_locations = np.cumsum([0, 365, 366, 365, 365, 365])
for i in (tick_locations):
    ax.vlines(i, 0, 2,linestyles='dotted', colors = 'grey')
    
ax.set_xticks(tick_locations)
ax.set_xticklabels([2015, 2016, 2017, 2018, 2019, 2020], rotation = 0)
ax.set_yticklabels(labels=[])
plt.ylim([0.45, 0.60])
ax.legend(loc="upper left", title="data")

plt.show()

##### Having four complete years available allows various types of testing and modelling. In this EDA we will limit to use the last year available (2018) as an hold-out, setting our baseline model to be able to forecast an entire year in the future.

##### As a last check we verify that no date is missing from train and test:

In [ ]:
missing_train = pd.date_range(start=train_dates.min(), end=train_dates.max()).difference(train_dates)
missing_test = pd.date_range(start=test_dates.min(), end=test_dates.max()).difference(test_dates)
print(f"missing dates in train: {len(missing_train)} and in test: {len(missing_test)}")

##### Having completed the checks, we process the datetime information, extracting it informative elements at different time granularities:

In [ ]:
# We create different time granularity

def process_time(df):
    df['year'] = df['date'].dt.year
    df['month'] = df['date'].dt.month
    df['week'] = df['date'].dt.isocalendar().week
    df['week'][df['week']>52] = 52
    df['day'] = df['date'].dt.day
    df['dayofweek'] = df['date'].dt.dayofweek
    df['quarter'] = df['date'].dt.quarter
    df['dayofyear'] = df['date'].dt.dayofyear
    return df

train = process_time(train)
test = process_time(test)

##### We are ready to explore the data. In order to highlight the time series characteristics, we create panels of products x countries x shops. We start by aggregating at a year level.

In [ ]:
for product in ['Kaggle Mug', 'Kaggle Hat', 'Kaggle Sticker']:
    print(f"\n--- {product} ---\n")
    fig = plt.figure(figsize=(20, 10), dpi=100)
    fig.subplots_adjust(hspace=0.25)
    for i, store in enumerate(['KaggleMart', 'KaggleRama']):
        for j, country in enumerate(['Finland', 'Norway', 'Sweden']):
            ax = fig.add_subplot(2, 3, (i*3+j+1))
            selection = (train['country']==country)&(train['store']==store)&(train['product']==product)
            selected = train[selection]
            selected.set_index('date').groupby('year')['num_sold'].mean().plot(ax=ax)
            ax.set_title(f"{country}:{store}")
    plt.show()

##### The first series of panels points out that the country effect is kind of indipendent from store and product. There is an underlying country dynamic that replicates the same no matter the shop or the product sold by it. We also notice that shops differentiate only for the level of sales.

##### Our next panels will explore seaasonality based on months:

In [ ]:
for product in ['Kaggle Mug', 'Kaggle Hat', 'Kaggle Sticker']:
    fig = plt.figure(figsize=(20, 10), dpi=100)
    fig.subplots_adjust(hspace=0.25)
    for i, store in enumerate(['KaggleMart', 'KaggleRama']):
        for j, country in enumerate(['Finland', 'Norway', 'Sweden']):
            ax = fig.add_subplot(2, 3, (i*3+j+1))
            selection = (train['country']==country)&(train['store']==store)&(train['product']==product)
            selected = train[selection]
            for year in [2015, 2016, 2017, 2018]:
                selected[selected.year==year].set_index('date').groupby('month')['num_sold'].mean().plot(ax=ax, label=year)
            ax.set_title(f"{product} | {country}:{store}")
            ax.legend()
    plt.show()

##### Here we notice two important elements: seanality curves are different for each product and they also differ from year to year. Averaging the curves probably is safe bet for the future, as well as considering more relevant the recent years (thus weighting more the year 2018 for instance). For the sticker product, year 2017 seems particularly different from others.

##### We now proceed to examine seasonality even more in detail at a week level:

In [ ]:
for product in ['Kaggle Mug', 'Kaggle Hat', 'Kaggle Sticker']:
    print(f"\n--- {product} ---\n")
    fig = plt.figure(figsize=(20, 10), dpi=100)
    fig.subplots_adjust(hspace=0.25)
    for i, store in enumerate(['KaggleMart', 'KaggleRama']):
        for j, country in enumerate(['Finland', 'Norway', 'Sweden']):
            ax = fig.add_subplot(2, 3, (i*3+j+1))
            selection = (train['country']==country)&(train['store']==store)&(train['product']==product)
            selected = train[selection]
            for year in [2015, 2016, 2017, 2018]:
                selected[selected.year==year].set_index('date').groupby('week')['num_sold'].mean().plot(ax=ax, label=year)
            ax.set_title(f"{country}:{store}")
            ax.legend()
    plt.show()

##### At a week level we see that differences are due to peaks. Peaks seem different in Spring. Probably is is Easter effect.

##### We now start obeserving recurrences at a monthly level:

In [ ]:
for product in ['Kaggle Mug', 'Kaggle Hat', 'Kaggle Sticker']:
    print(f"\n--- {product} ---\n")
    fig = plt.figure(figsize=(20, 10), dpi=100)
    fig.subplots_adjust(hspace=0.25)
    for i, store in enumerate(['KaggleMart', 'KaggleRama']):
        for j, country in enumerate(['Finland', 'Norway', 'Sweden']):
            ax = fig.add_subplot(2, 3, (i*3+j+1))
            selection = (train['country']==country)&(train['store']==store)&(train['product']==product)
            selected = train[selection]
            for year in [2015, 2016, 2017, 2018]:
                selected[selected.year==year].set_index('date').groupby('day')['num_sold'].mean().plot(ax=ax, label=year)
            ax.set_title(f"{country}:{store}")
            ax.legend()
    plt.show()

##### The middle of the month usually presents less sales. The peak at the end may be influenced by seasonal peaks (end of year).

##### And we completed by inspecting at a day of the week level:

In [ ]:
for product in ['Kaggle Mug', 'Kaggle Hat', 'Kaggle Sticker']:
    print(f"\n--- {product} ---\n")
    fig = plt.figure(figsize=(20, 10), dpi=100)
    fig.subplots_adjust(hspace=0.25)
    for i, store in enumerate(['KaggleMart', 'KaggleRama']):
        for j, country in enumerate(['Finland', 'Norway', 'Sweden']):
            ax = fig.add_subplot(2, 3, (i*3+j+1))
            selection = (train['country']==country)&(train['store']==store)&(train['product']==product)
            selected = train[selection]
            for year in [2015, 2016, 2017, 2018]:
                selected[selected.year==year].set_index('date').groupby('dayofweek')['num_sold'].sum().plot(ax=ax, label=year)
            ax.set_title(f"{country}:{store}")
            ax.legend()
    plt.show()

##### Friday and the week-end are the best days, but Sundays are not always at the same level as Saturdays (it depends on the year - why?).

##### Based on the information we got we now proceed to feature engineering and to enrich the data (using festivities and GDP data).

In [ ]:
festivities = pd.read_csv("../input/festivities-in-finland-norway-sweden-tsp-0122/nordic_holidays.csv",
                          parse_dates=['date'],
                          usecols=['date', 'country', 'holiday'])

In [ ]:
gdp = pd.read_csv("../input/gdp-20152019-finland-norway-and-sweden/GDP_data_2015_to_2019_Finland_Norway_Sweden.csv")
gdp = np.concatenate([gdp[['year', 'GDP_Finland']].values, 
                      gdp[['year', 'GDP_Norway']].values, 
                      gdp[['year', 'GDP_Sweden']].values])
gdp = pd.DataFrame(gdp, columns=['year', 'gdp'])
gdp['country'] = ['Finland']*5 + ['Norway']*5 +['Sweden']*5

##### We now process the data and scale it. Since EDA revealed how the different characteristics of the series are mostly main effects (country and store), we focus on finding the way to model the interaction between products and time.

In [ ]:
train['product'].unique(), train['store'].unique()

In [ ]:
from sklearn.preprocessing import StandardScaler, MinMaxScaler, RobustScaler

def process_data(df):
    
    processed = dict()
    processed['row_id'] = df['row_id']
    
    print("creating dummies for main effects of time, country, store and product")
    to_dummies = ['country', 'store', 'product', 'month', 'week', 'day', 'dayofweek']
    for feat in to_dummies:
        tmp = pd.get_dummies(df[feat])
        for col in tmp.columns:
            processed[feat+'_'+str(col)] = tmp[col]
    
    print("creating dummies with 7 gg halo effect for Nordic holidays")
    tmp = pd.get_dummies(
        df.merge(festivities, on=['date', 'country'], how='left').sort_values('row_id')['holiday'])
    for col in tmp.columns:
            peak = tmp[col].values + tmp[col].rolling(7).mean().fillna(0).values
            processed['holiday_'+str(col)] =  peak
    
    print("creating interactions")
    high_lvl_interactions = [
        ['country', 'product', 'month'],
        ['country', 'product', 'week'],
        ['country', 'store', 'week'],
        ['country', 'product', 'month', 'day'],
        ['country', 'product', 'month', 'dayofweek'],
    ]
    for sel in high_lvl_interactions:
        tmp = pd.get_dummies(df[sel].apply(lambda row: '_'.join(row.values.astype(str)), axis=1))
        for col in tmp.columns:
            processed[col] = tmp[col]
            
    print("modelling time as continuous per each country")
    for country in ['Finland', 'Norway', 'Sweden']:
        processed[country + '_prog'] = ((df.row_id // 18) + 1) * (df['country']==country).astype(int)
        processed[country + '_prog^2'] = (processed[country + '_prog']**2)
        processed[country + '_prog^3'] = (processed[country + '_prog']**3)
    
    print("adding sin/cos of day of year")
    dayofyear = df.date.dt.dayofyear
    steps = 4
    for k in range(1, 32, steps):
        processed[f'sin{k}'] = np.sin(dayofyear / 365 * 2 * np.pi * k)
        processed[f'cos{k}'] = np.cos(dayofyear / 365 * 2 * np.pi * k)
    
    print("adding log(gdp)")
    gdp_countries = np.log1p(df.merge(gdp, on=['country', 'year'], how='left')['gdp'].values)
    for country in ['Finland', 'Norway', 'Sweden']:
        processed['gdp_'+ country] = gdp_countries * (df['country']==country).astype(int)
            
    print(f"completed processing {len(processed)-1} features")
    
    values = list()
    columns = list()
    for key, value in processed.items():
        values.append(np.array(value))
        columns.append(key)
        
    values = np.array(values).T        
    processed = pd.DataFrame(values, columns=columns)
    
    print("resorting row ids")
    processed = processed.sort_values('row_id').set_index('row_id')
    return processed

def process_target(df):
    target = pd.DataFrame({'row_id':df['row_id'], 'num_sold':df['num_sold']})
    target = target.sort_values('row_id').set_index('row_id')
    return target

train_test = process_data(train.append(test))

processed_train = train_test.iloc[:len(train)].copy()
processed_test = train_test.iloc[len(train):].copy()

target = np.ravel(process_target(train))

##### Since EDA demonstrated that there is some kind of yearly change, we overweight more recent observations.

In [ ]:
def weighting(df, weights):
    return df.year.replace(weights).values
    
weights = weighting(train, {2015:0.125, 2016:0.25, 2017:0.5, 2018:1})

In [ ]:
processed_train.shape, train.shape, processed_test.shape, test.shape

##### We prepare all the evaluation measures, both at an aggregate level, with exp transformation and at an individual cases level (for error analysis)

In [ ]:
def SMAPE(y_true, y_pred):
    # From https://www.kaggle.com/cpmpml/smape-weirdness
    denominator = (y_true + np.abs(y_pred)) / 200.0
    diff = np.abs(y_true - y_pred) / denominator
    diff[denominator == 0] = 0.0
    return np.mean(diff)

def SMAPE_exp(y_true, y_pred):
    y_true = np.exp(y_true)
    y_pred = np.exp(y_pred)
    denominator = (y_true + np.abs(y_pred)) / 200.0
    diff = np.abs(y_true - y_pred) / denominator
    diff[denominator == 0] = 0.0
    return np.mean(diff)

def SMAPE_err(y_true, y_pred):
    # From https://www.kaggle.com/cpmpml/smape-weirdness
    denominator = (y_true + np.abs(y_pred)) / 200.0
    diff = np.abs(y_true - y_pred) / denominator
    diff[denominator == 0] = 0.0
    return diff

##### We try modelling both an additive and a multiplicative predictor (for multiplicative you just need a log transoformation of the target because exponentiation of the sum of logs of the terms correspond to multiplications of the terms)

In [ ]:
# due to float calculations the computation is approximated
a = 5
b = 7
print(a * b) # pure multiplicative
print(np.exp(np.log(a) + np.log(b))) # multiplicative made additive by log

In [ ]:
from sklearn.linear_model import LinearRegression

addictive_model = LinearRegression().fit(processed_train, target, sample_weight=weights)
addictive_fit = addictive_model.predict(processed_train.values)
print(f"addictive fit: {SMAPE(y_true=target, y_pred=addictive_fit): 0.3f}")

In [ ]:
multiplicative_model = LinearRegression().fit(processed_train, np.log(target), sample_weight=weights)
multiplicative_fit = np.exp(multiplicative_model.predict(processed_train.values))
print(f"multiplicative fit: {SMAPE(y_true=target, y_pred=multiplicative_fit):0.3f}")

##### It seems that the multiplicative model is better. We now use it to predict on the test set.

In [ ]:
def selective_rounding(preds, lower=0.3, upper=0.7):
    # selective rounding
    dec = preds % 1
    to_round = (dec<=lower)|(dec>=upper)
    preds[to_round] = np.round(preds[to_round])
    return preds

In [ ]:
model = LinearRegression().fit(processed_train, np.log(target), sample_weight=weights)
submission = pd.read_csv("../input/tabular-playground-series-jan-2022/sample_submission.csv")
preds = np.exp(model.predict(processed_test.values))

# rounding
preds = selective_rounding(preds, lower=0.3, upper=0.7)

submission.num_sold = preds
submission.to_csv("submission.csv", index=False)

##### For testing purposes we refit the model only on years 2015-2017 using 2018 as an hold-out test.

In [ ]:
log_target = np.log(target)
train_set = list(train.index[train.date<'2018-01-01'])
val_set = list(train.index[train.date>='2018-01-01'])
model = LinearRegression()

model.fit(processed_train.iloc[train_set], log_target[train_set], sample_weight=weights[train_set])
preds = np.exp(model.predict(processed_train.iloc[val_set].values))

# rounding
preds = selective_rounding(preds, lower=0.3, upper=0.7)

smape = SMAPE(y_true=target[val_set], y_pred=preds)
print(f"hold-out smape: {smape}")

### If you liked the notebook, consider to upvote, thank you and happy Kaggling!